In [1]:
import os
import numpy as np
from scipy.stats import f
import pandas as pd
import pingouin as pg

In [2]:
#strict consistency
path_no_context_strict_consistency_chatgpt = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/ChatGPT/strict_consistency/'
path_no_context_strict_consistency_googlebard = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/GoogleBard/strict_consistency/'
path_context_strict_consistency_chatgpt = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/context/ChatGPT/strict_consistency/'

#soft consistency
path_no_context_soft_consistency_chatgpt = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/ChatGPT/soft_consistency/'
path_no_context_soft_consistency_googlebard = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/GoogleBard/soft_consistency/'
path_context_soft_consistency_chatgpt = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/context/ChatGPT/soft_consistency/'

In [3]:
def transpose_df(directory_path, filename):
    df = pd.read_csv(os.path.join(directory_path, filename))
    df.drop(columns=['run_index'], inplace=True) if 'run_index' in df.columns else None
    df.drop(columns=['ID'], inplace=True) if 'ID' in df.columns else None

    # Swap the x-axis with the y-axis
    df_transposed = df.transpose()

    return df_transposed

In [4]:
def calculate_cronbach_alpha(df_transposed):
    cronbach_alpha = pg.cronbach_alpha(data=df_transposed)
    
    return cronbach_alpha

In [5]:
def iterate_through_files_in_directory(directory_path):
    filenames = os.listdir(directory_path)
    
    result_df = pd.DataFrame()

    for filename in filenames:
        
        if filename.endswith('.csv'):
            #transpose dataframe
            df_transposed = transpose_df(directory_path, filename)

            # Replace strings with NaN
            df_transposed = df_transposed.map(lambda x: np.nan if isinstance(x, str) else x)

            #calculate cronbach alpha
            cronbach_alpha = calculate_cronbach_alpha(df_transposed)

            # Create a Series
            s = pd.Series({'dataset': str(filename), '/'.join(directory_path.split('/')[-4:]): cronbach_alpha[0]})

            # Convert the Series to a DataFrame
            df = s.to_frame().T

            # Add values to the DataFrame
            result_df = pd.concat([result_df, df], ignore_index=True)
        
    return result_df

In [6]:
# results for strict consistency
result_df_no_context_chatgpt_strict_consistency = iterate_through_files_in_directory(path_no_context_strict_consistency_chatgpt)
result_df_no_context_googlebard_strict_consistency = iterate_through_files_in_directory(path_no_context_strict_consistency_googlebard)
result_df_context_chatgpt_strict_consistency = iterate_through_files_in_directory(path_context_strict_consistency_chatgpt)

/Users/brain/anaconda3/lib/python3.10/site-packages/pingouin/reliability.py:144: RuntimeWarning: invalid value encountered in scalar divide
  cronbach = (k / (k - 1)) * (1 - np.trace(C) / C.sum().sum())


In [7]:
# results for soft consistency
result_df_no_context_chatgpt_soft_consistency = iterate_through_files_in_directory(path_no_context_soft_consistency_chatgpt)
result_df_no_context_googlebard_soft_consistency = iterate_through_files_in_directory(path_no_context_soft_consistency_googlebard)
result_df_context_chatgpt_soft_consistency = iterate_through_files_in_directory(path_context_soft_consistency_chatgpt)

/Users/brain/anaconda3/lib/python3.10/site-packages/pingouin/reliability.py:144: RuntimeWarning: invalid value encountered in scalar divide
  cronbach = (k / (k - 1)) * (1 - np.trace(C) / C.sum().sum())


In [14]:
#merge result dataframes
merged_df = pd.concat([
    result_df_no_context_chatgpt_strict_consistency.set_index('dataset'), 
    result_df_no_context_googlebard_strict_consistency.set_index('dataset'), 
    result_df_context_chatgpt_strict_consistency.set_index('dataset'), 
    result_df_no_context_chatgpt_soft_consistency.set_index('dataset'),
    result_df_no_context_googlebard_soft_consistency.set_index('dataset'),
    result_df_context_chatgpt_soft_consistency.set_index('dataset')],axis=1)

In [29]:
merged_df.to_csv('/Users/brain/Documents/GitHub/LLMs-topic-classification/results/cronbach_alpha.csv', index=False)